# Clean Scraped data from SCMP

This notebook runs through the data we just and cleans it up so that we can explore it properly in the next notebook. Separating null values from complete rows, splitting some of the data into more usable segments, and converting some of the columns into more usable variable types. 

## Import

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', 30)

import numpy as np 
import os

In [2]:
hk_csv = pd.read_csv('hkhorses.csv')
hk_csv.sample(3)

,name,sire,dame,date,race_number,track,distance,cl,rank,trainer,weight,jockey,dr,gr,win_time,last_qtr,section_time,ln_running,w_m,horse_wt,rt,odds_on,odds_last
13992,SMART BABY (A160) 型酒杯,Shamardal,Marine City,02-04-17,541,ST tf g/f B+2,1400,3,14,C. Fownes,126,C. Y. Ho,13,,1:21.9,23.0,36.54 23.28 24.37,13-14-14,9.25,1212,73,67,99
986,BLAZING PARTNERS (B071) 晴朗拍檔,Echoes Of Heaven,Flightime,02-02-19,386,ST tf g C,1000,4,11,D. Ferraris,125,N. Callan,6,,0:56:93,22.8,13.86 21.03 23.41,13-14-11,8.5,1073,50,89,87
1288,BUFFERING (T604) 緩衝之計,Mossman,Action Annie,01-05-16,607,ST tf g B,1200,G1,14,R. Heathcote,126,D. Browne,3,B,1:08.6,23.2,23.34 22.23 25.12,1-2-14,12.5,1130,118,41,13


Check our data for any null values we may have to move/replace to make sure our data is complete. 

In [3]:
hk_csv.isnull().any()

name            False
sire            False
dame            False
date            False
race_number     False
track           False
distance        False
cl              False
rank            False
trainer         False
weight          False
jockey          False
dr              False
gr              False
win_time         True
last_qtr         True
section_time    False
ln_running      False
w_m             False
horse_wt        False
rt              False
odds_on         False
odds_last       False
dtype: bool

In [4]:
nulls = hk_csv.loc[hk_csv['win_time'].isnull() | hk_csv['last_qtr'].isnull()]
nulls

,name,sire,dame,date,race_number,track,distance,cl,rank,trainer,weight,jockey,dr,gr,win_time,last_qtr,section_time,ln_running,w_m,horse_wt,rt,odds_on,odds_last
1252,BREEDERS' STAR (V197) 育馬妙星,Jet Spur,Like The Wind,26-10-16,134,HV tf g/f C+3,1000,4,DNF,D. Ferraris,128,D. Whyte,1,H,NaN,NaN,,6-6-3,2.5,1161,54,8.2,
2471,D B PIN (V344) 翩翩,Darci Brahma,Pins 'n' Needles,26-10-16,134,HV tf g/f C+3,1000,4,DNF,John Size,130,Z. Purton,4,,NaN,NaN,,2-2-2,SH,1142,56,2.5,
4343,FLYING MONKEY (T361) 勁飛聖,Pins,Dandify,26-10-16,134,HV tf g/f C+3,1000,4,DNF,W. Y. So,133,J. Moreira,11,TT,NaN,NaN,,1-1-2,1.25,1079,59,10,
5288,GOLDEN GLORY (T076) 金碧輝煌,Patapan,Victoria Bay,26-10-16,134,HV tf g/f C+3,1000,4,DNF,A. Lee,119,K. Teetan,8,B,NaN,NaN,,8-5-12,5.5,1065,45,27,
7946,JUMBO HAPPINESS (V315) 同進,Fastnet Rock,Tashzara,26-10-16,134,HV tf g/f C+3,1000,4,DNF,C. Fownes,129,C. Y. Ho,2,,NaN,NaN,,12-11-5,2,1086,57,10,
15882,THE FULL BLOOM (V243) 盛勢,Zebedee,Break Of Dawn,26-10-16,134,HV tf g/f C+3,1000,4,DNF,C. H. Yip,129,C. Schofield,6,,NaN,NaN,,7-8-7,2,1052,55,18,


Showing all DNF rows shows all above null values as well as a few additional rows.  All of these values make sense, and while not inherently inaccurate, they will be separated into their own csv file later in this notebook in order to ensure we can analyze finished races and unfinished races separately.  

In [5]:
dnfs = hk_csv.loc[hk_csv['rank'] == 'DNF']
dnfs

,name,sire,dame,date,race_number,track,distance,cl,rank,trainer,weight,jockey,dr,gr,win_time,last_qtr,section_time,ln_running,w_m,horse_wt,rt,odds_on,odds_last
1252,BREEDERS' STAR (V197) 育馬妙星,Jet Spur,Like The Wind,26-10-16,134,HV tf g/f C+3,1000,4,DNF,D. Ferraris,128,D. Whyte,1,H,NaN,NaN,,6-6-3,2.5,1161,54,8.2,
2471,D B PIN (V344) 翩翩,Darci Brahma,Pins 'n' Needles,26-10-16,134,HV tf g/f C+3,1000,4,DNF,John Size,130,Z. Purton,4,,NaN,NaN,,2-2-2,SH,1142,56,2.5,
4343,FLYING MONKEY (T361) 勁飛聖,Pins,Dandify,26-10-16,134,HV tf g/f C+3,1000,4,DNF,W. Y. So,133,J. Moreira,11,TT,NaN,NaN,,1-1-2,1.25,1079,59,10,
5288,GOLDEN GLORY (T076) 金碧輝煌,Patapan,Victoria Bay,26-10-16,134,HV tf g/f C+3,1000,4,DNF,A. Lee,119,K. Teetan,8,B,NaN,NaN,,8-5-12,5.5,1065,45,27,
7946,JUMBO HAPPINESS (V315) 同進,Fastnet Rock,Tashzara,26-10-16,134,HV tf g/f C+3,1000,4,DNF,C. Fownes,129,C. Y. Ho,2,,NaN,NaN,,12-11-5,2,1086,57,10,
9705,MARQULA (V423) 駿王,Coast Guard,Marquet Formula,14-04-19,580,ST tf g/y C,1600,4,DNF,C. W. Chang,131,N. Callan,9,B/TT,1:36:63,25.15,,6-7-6-9,6,1230,56,19,64
10618,MR WEALTH (B265) 發財先生,Tamayuz,Empire Rose,24-03-19,529,ST tf g C+3,1400,3,DNF,K. L. Man,125,N. Callan,9,H/P,1:22:74,23.43,14.25 22.36 23.98,10-9-8-9,3.75,1145,72,11,24
15882,THE FULL BLOOM (V243) 盛勢,Zebedee,Break Of Dawn,26-10-16,134,HV tf g/f C+3,1000,4,DNF,C. H. Yip,129,C. Schofield,6,,NaN,NaN,,7-8-7,2,1052,55,18,


Take the entire scrapped 'name' string from HKRacing, and break it down to its 3 fundamental parts: the horse's English name, its ID, and its name in Chinese, giving each a column of their own. The most important part obviously being the ID, however since we have both the English and Chinese recorded, it will be recorded anyways in the event anyone wants to use them. 

In [6]:
hk_clean = hk_csv.copy()

hk_clean[['english_name', 'ID']] = hk_clean['name'].str.split(' \(', n=1, expand=True)
hk_clean[['ID', 'chinese_name']] = hk_clean['ID'].str.split('\)', n=1, expand=True)

cols = list(hk_clean.columns.values)[-3:] + list(hk_clean.columns.values)[:-3]
cols.remove('name')
hk_clean = hk_clean[cols]

hk_clean.sample(3)

,english_name,ID,chinese_name,sire,dame,date,race_number,track,distance,cl,rank,trainer,weight,jockey,dr,gr,win_time,last_qtr,section_time,ln_running,w_m,horse_wt,rt,odds_on,odds_last
13319,ROUNDABOUT,T352,飛星凱旋,I Am Invincible,Fidele,10-06-18,721,ST tf g/f C,1200,3,3,A. T. Millard,120,C. Wong,1,SR,1:08:76,22.64,24.40 22.32 22.33,7-8-3,2,1129,76,6.8,3.8
9194,LOTUS BREEZE,T112,麯院風荷,Oasis Dream,Ebadiyla,02-05-18,617,ST aw g -,1200,4,6,W. Y. So,133,C. Schofield,6,CP/XB/TT,1:09:25,22.63,24.22 22.76 22.94,3-4-6,4.25,1087,59,12,14
393,ALL BEST FRIENDS,A217,好友無敵,Street Cry,Ambitious Lady,23-12-17,297,ST aw g -,1650,3,9,K. L. Man,117,W. M. Lai,12,,1:37:86,23.79,51.36 23.15 24.60,2-2-3-9,7.75,1071,61,67,99


We'll immediately remove the chinese name for each horse (as I don't speak Chinese), however I wanted to make it its own cell so this step could be easily skipped by someone who wanted to use them for whatever reason.  

In [7]:
hk_clean.drop("chinese_name", axis=1, inplace=True)
hk_clean.sample(3)

,english_name,ID,sire,dame,date,race_number,track,distance,cl,rank,trainer,weight,jockey,dr,gr,win_time,last_qtr,section_time,ln_running,w_m,horse_wt,rt,odds_on,odds_last
5805,GRACIOUS RYDER,A179,Tickets,Fleur De Lion,22-04-19,595,ST tf g C+3,1400,4,14,D. Hall,120,M. F. Poon,2,B,1:21:92,23.37,35.97 23.30 24.48,6-8-11-14,11.5,1092,49,21,23
13939,SMART CHARADE,A314,Fastnet Rock,Aniarnota,27-05-18,678,ST tf g A,1600,4,4,W. Y. So,129,A. Sanna,11,B/TT,1:35:98,22.06,49.80 24.20 22.24,6-4-3-4,1.75,1115,56,10,8.8
5991,GREAT TREASURE,T380,Husson,Russian Belle,18-11-18,183,ST aw g -,1650,3,2,Y. S. Tsui,121,S. De Sousa,3,H-/B,1:39:03,24.24,52.18 23.09 23.83,7-7-4-2,0.5,1010,66,10,11


Now we check what columns have what data types.  For columns that will need to be considered numbers for future analysis, check what unique characters is causing Pandas to not auto convert the column into an integer or float. 

In [8]:
hk_clean.dtypes

english_name     object
ID               object
sire             object
dame             object
date             object
race_number       int64
track            object
distance          int64
cl               object
rank             object
trainer          object
weight            int64
jockey           object
dr                int64
gr               object
win_time         object
last_qtr        float64
section_time     object
ln_running       object
w_m              object
horse_wt         object
rt               object
odds_on          object
odds_last        object
dtype: object

In [9]:
hk_clean['rank'].unique()

array(['10', '1', '11', '5', '4', '3', '13', '6', '9', '2', '12', '7',
       '8', '14', 'WD', 'DNF', 'UR', 'DISQ'], dtype=object)

In [10]:
hk_clean['horse_wt'].unique()

array(['1134', '1111', '1043', '1033', '1027', '1036', '1047', '1066',
       '1064', '1055', '1058', '1051', '1048', '1042', '1053', '1054',
       '1062', '1025', '1035', '1044', '1041', '1021', '1005', '1002',
       '1011', '1020', '1024', '1045', '1040', '1037', '1032', '1031',
       '1034', '1026', '1017', '1004', '1014', '1013', '1142', '1146',
       '1155', '1156', '1174', '1166', '1168', '1152', '1158', '1159',
       '1150', '1151', '1157', '1140', '1129', '1130', '1138', '1143',
       '1145', '1144', '1133', '1097', '1105', '1090', '1098', '1117',
       '1114', '1104', '1125', '1122', '1121', '1240', '1214', '1212',
       '1209', '1197', '1171', '1162', '1093', '1089', '1096', '1248',
       '1244', '1234', '1223', '1256', '1267', '1247', '1246', '1007',
       '1009', '1181', '1188', '1186', '1187', '1203', '1215', '1207',
       '1206', '1185', '1173', '1179', '1192', '1202', '1204', '1107',
       '1088', '1103', '1069', '1080', '1108', '1109', '1110', '1091',
      

In [11]:
hk_clean['rt'].unique()

array(['129', '114', '94', '34', '35', '37', '39', '40', '42', '44', '43',
       '46', '48', '50', '52', '59', '57', '60', '62', '64', '66', '67',
       '61', '63', '56', '58', '68', '70', '71', '73', '74', '69', '47',
       '36', '41', '45', '51', '53', '55', '--', '49', '72', '65', '54',
       '118', '76', '75', '77', '79', '78', '80', '81', '83', '85', '87',
       '26', '28', '29', '31', '33', '38', '82', '30', '25', '113', '111',
       '89', '91', '92', '127', '125', '120', '110', '108', '99', '116',
       '137', '126', '132', '119', '112', '117', '106', '100', '93', '84',
       '86', '32', '88', '90', '109', '107', '103', '102', '97', '96',
       '22', '18', '19', '21', '20', '24', '95', '23', '104', '98', '27',
       '115', '17', '121', '123', '130', '128', '122', '124', '101',
       '105', '16', '10', '5', '7', '13', '15'], dtype=object)

In [12]:
hk_clean['odds_on'].unique()

array(['6.8', '28', '3.8', '9.1', '12', '7.1', '9.3', '6.2', '22', '7.3',
       '14', '36', '57', '24', '95', '72', '67', '13', '15', '29', '44',
       '30', '27', '31', '10', '16', '9.7', '5.6', '8.9', '4.7', '23',
       '21', '33', '6.6', '7', '4.9', '5.1', '50', '4.5', '39', '20',
       '42', '46', '35', '9.9', '11', '9.8', '8.2', '1.9', '5.2', '8.4',
       '4.2', '18', '73', '25', '1.8', '5.4', '2.8', '4.4', '5.7', '3.3',
       '3.2', '2.3', '2.5', '2.2', '1.6', '2.1', '\xa0', '79', '78',
       '6.3', '6.4', '8', '6.7', '8.6', '7.8', '19', '32', '52', '89',
       '87', '69', '62', '64', '96', '8.1', '6.9', '4', '3.5', '3.4',
       '4.6', '9', '77', '17', '53', '9.2', '40', '6', '7.9', '3.6',
       '4.8', '45', '65', '34', '38', '58', '37', '80', '7.6', '8.3',
       '9.5', '81', '61', '70', '7.7', '9.6', '9.4', '2', '51', '7.4',
       '6.1', '5.8', '1.7', '8.7', '5.5', '8.8', '26', '63', '48', '55',
       '2.7', '43', '56', '7.5', '47', '99', '6.5', '54', '49', '84',
  

In [13]:
hk_clean['odds_last'].unique()

array(['8.6', '38', '7.1', '2.7', '12', '8.7', '6.3', '7.3', '37', '8.8',
       '14', '25', '64', '99', '15', '24', '16', '62', '13', '30', '36',
       '19', '7.6', '6.9', '7.2', '17', '11', '4.9', '43', '7.9', '18',
       '53', '20', '9.3', '5.4', '5.8', '9.6', '78', '4.8', '35', '45',
       '32', '46', '55', '77', '7.7', '75', '26', '29', '8.5', '6.4',
       '2.5', '4.1', '6.6', '65', '93', '79', '40', '42', '51', '1.3',
       '8.4', '5.7', '9.9', '2.8', '4.6', '3.6', '2.2', '3', '2.9', '1.9',
       '1.7', '10', '\xa0', '4.3', '7.4', '6.2', '3.7', '5.2', '2.3',
       '2.1', '3.1', '3.5', '7.5', '21', '9.5', '8.2', '22', '98', '39',
       '66', '67', '3.3', '3.8', '3.4', '3.2', '28', '69', '6.5', '4.4',
       '56', '59', '7', '7.8', '8.1', '5', '47', '4.5', '3.9', '9', '4.7',
       '86', '50', '9.4', '5.6', '41', '33', '6.7', '8', '9.8', '4.2',
       '63', '9.2', '5.1', '23', '1.8', '1.5', '6.8', '4', '52', '34',
       '31', '27', '44', '6', '9.1', '84', '5.5', '2', '74',

After checking for odd unique values above, you can see that for blank cells, SCMP uses an odd non-breaking blank space encoding on their website, so we simply have to replace the '\xa0' blank character they use with an empty space before we go about converting the columns we want to integers. We also need to replace any instances of '--' that the website sometimes uses in place of a missing value before converting the column. 

Cannot convert the 'rank' column, as there are some values such as 'WD' that aren't mere placeholders, but aren't integers.  These unfinished races will be removed below and placed in their own csv file to allow the rank to be properly converted. 

In [14]:
#If statement here to prevent errors from running this cell multiple times, making it try and convert strings that don't exist anymore. 
if hk_clean['horse_wt'].dtypes != float or hk_clean['rt'].dtypes != float or hk_clean['odds_on'].dtypes != float or hk_clean['odds_last'].dtypes != float: 
    hk_clean.horse_wt.replace(['--'], [""], inplace=True)
    hk_clean.rt.replace(['--'], [""], inplace=True)
    hk_clean.odds_on.replace(['\xa0'], [""], inplace=True)
    hk_clean.odds_last.replace(['\xa0'], [""], inplace=True)
    hk_clean[["horse_wt", "rt", "odds_on", "odds_last"]] = hk_clean[["horse_wt", "rt", "odds_on", "odds_last"]].apply(pd.to_numeric)
    

#While converting date to datetime, making it convert to a dayfirst format isn't strict due to a bug in Pandas, but should function fine for these simple dates. 
hk_clean['date'] = pd.to_datetime(hk_clean['date'], dayfirst=True) 
hk_clean.dtypes

english_name            object
ID                      object
sire                    object
dame                    object
date            datetime64[ns]
race_number              int64
track                   object
distance                 int64
cl                      object
rank                    object
trainer                 object
weight                   int64
jockey                  object
dr                       int64
gr                      object
win_time                object
last_qtr               float64
section_time            object
ln_running              object
w_m                     object
horse_wt               float64
rt                     float64
odds_on                float64
odds_last              float64
dtype: object

We also want to convert 'rank', however it has some non-integer numbers.  WD = pulled out before going to the start, DNF = Did not Finish, UR = Unseated Rider, DISQ = Disqualified.  While none of these results are incorrect values, they are also not going to be of much use to us for analyzing races that have successfully finished.  We also want to remove these values to allow us to properly compare ranks of different races, as that is some of the most crucial data we have. 

In the event that we want to analyze why races end in absolute failure (not just coming last) however, we'll create a dataframe containing all of these results so they can be analyzed. 

In [15]:
dnf_list = ('WD', 'DNF', 'UR', 'DISQ')

finished_hk_clean = hk_clean[~hk_clean['rank'].isin(dnf_list)]
dnf_hk_clean = hk_clean[hk_clean['rank'].isin(dnf_list)]
print(finished_hk_clean['rank'].unique(), dnf_hk_clean['rank'].unique())

['10' '1' '11' '5' '4' '3' '13' '6' '9' '2' '12' '7' '8' '14'] ['WD' 'DNF' 'UR' 'DISQ']


Now that the integer values have been separated from the DNF values, we can convert the column to float values successfully and store finished and unfinished races separately. 

In [16]:
finished_hk_clean[["rank"]] = finished_hk_clean[["rank"]].apply(pd.to_numeric)
finished_hk_clean.dtypes

C:\Users\Dakota Owen\AppData\Local\conda\conda\envs\py37\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


english_name            object
ID                      object
sire                    object
dame                    object
date            datetime64[ns]
race_number              int64
track                   object
distance                 int64
cl                      object
rank                     int64
trainer                 object
weight                   int64
jockey                  object
dr                       int64
gr                      object
win_time                object
last_qtr               float64
section_time            object
ln_running              object
w_m                     object
horse_wt               float64
rt                     float64
odds_on                float64
odds_last              float64
dtype: object

In [17]:
finished_hk_clean.to_csv(os.getcwd() + '\\project_files\\finished_hkhorses.csv', index=False)
dnf_hk_clean.to_csv(os.getcwd() + '\\project_files\\dnf_hkhorses.csv', index=False)
print(finished_hk_clean.shape, dnf_hk_clean.shape)

(17303, 24) (146, 24)
